In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
 
from tqdm import tqdm 

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import os
from skimage.measure import label, regionprops
import cv2

In [ ]:
print(os.getcwd())

In [ ]:
TRAIN_PATH = '/home/kmit/Nuclear_Pleomorpism_Birac/BreCaHAD/train/images/'
TEST_PATH = '/home/kmit/Nuclear_Pleomorpism_Birac/BreCaHAD/test/images/'

train_ids = os.listdir(TRAIN_PATH)
test_ids = os.listdir(TEST_PATH)

In [ ]:

print(train_ids)

In [ ]:
IMG_WIDTH = 1360
IMG_HEIGHT = 1024
IMG_CHANNELS = 3
BATCH_SIZE = 32
seed = 42

In [ ]:
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
  print(id_)
  path = TRAIN_PATH 
  img = imread(path + id_)[:,:,:IMG_CHANNELS]  
  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_train[n] = img 
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
  mask_ = imread('/home/kmit/Nuclear_Pleomorpism_Birac/BreCaHAD/train/mask/'+ id_[:-4]+"_mask.png",as_gray=True)
  mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',  
                                    preserve_range=True), axis=-1)
  print(mask_.shape)
  mask_=np.reshape(mask_,(1024,1360,1))
  mask = np.maximum(mask, mask_)             
  Y_train[n] = mask

In [ ]:
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH 
    img = imread(path +  id_ )[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

In [ ]:
def draw_sub_plots(n,m,X,Y):
  f, axarr = plt.subplots(n,m,figsize=((10, 10)))
  ind=0
  for row in range(n):
    for col in range(m):
      if(col%2==0):
        img = X[row].reshape((1024,1360,3))
        axarr[row, 0].axis("off")
        axarr[row,0].imshow(img)
        title = "Train Image "+str(row+1)
        title_color = 'black'
        title = axarr[row, 0].set_title(title)
        plt.setp(title, color=title_color)
    else:
      img = Y[row]
      axarr[row, 1].axis("off")
      axarr[row,1].imshow(img)
      title = "Ground Truth Image "+str(row+1)
      title_color = 'black'
      title = axarr[row, 1].set_title(title)
      plt.setp(title, color=title_color)

In [ ]:
X=[]
Y=[]
for ind in range(4):
  i = random.randint(0, len(train_ids))
  X.append(X_train[i])
  Y.append(np.squeeze(Y_train[i]))

In [ ]:
draw_sub_plots(4,2,X,Y)

In [ ]:
from keras.preprocessing import image

image_datagen = image.ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='reflect')
mask_datagen = image.ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='reflect')

image_datagen.fit(X_train[:int(X_train.shape[0]*0.9)], augment=True, seed=seed)
mask_datagen.fit(Y_train[:int(Y_train.shape[0]*0.9)], augment=True, seed=seed)

x=image_datagen.flow(X_train[:int(X_train.shape[0]*0.9)],batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y=mask_datagen.flow(Y_train[:int(Y_train.shape[0]*0.9)],batch_size=BATCH_SIZE,shuffle=True, seed=seed)

image_datagen_val = image.ImageDataGenerator()
mask_datagen_val = image.ImageDataGenerator()

image_datagen_val.fit(X_train[int(X_train.shape[0]*0.9):], augment=True, seed=seed)
mask_datagen_val.fit(Y_train[int(Y_train.shape[0]*0.9):], augment=True, seed=seed)

x_val=image_datagen_val.flow(X_train[int(X_train.shape[0]*0.9):],batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y_val=mask_datagen_val.flow(Y_train[int(Y_train.shape[0]*0.9):],batch_size=BATCH_SIZE,shuffle=True, seed=seed)

In [ ]:
imshow(x.next()[0].astype(np.uint8))
plt.show()
imshow(np.squeeze(y.next()[0]))
plt.show()

In [ ]:
train_generator = zip(x, y)
val_generator = zip(x_val, y_val)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience=3, verbose=1)
checkpointer = ModelCheckpoint('model_unet_brecahd-1.h5', verbose=1, save_best_only=True)

In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
model.summary()

In [ ]:
results = model.fit_generator(train_generator, validation_data=val_generator, validation_steps=10, steps_per_epoch=250,
                              epochs=10, callbacks=[earlystopper, checkpointer])

In [ ]:
from keras.models import Model, load_model
model = load_model('model_unet_brecahd-1.h5')
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
def draw_sub_plots_3(n,m,X,Y,Pred,name=True):
  f, axarr = plt.subplots(n,m,figsize=((15, 15)))
  ind=0
  for row in range(n):
    for col in range(m):
      if(col==0):
        img = X[row].reshape((1024,1360,3))
        axarr[row, 0].axis("off")
        axarr[row,0].imshow(img)
        title = "Train Image "+str(row+1)
        title_color = 'black'
        title = axarr[row, 0].set_title(title)
        plt.setp(title, color=title_color)
      
      elif(col==1):
        img = Y[row]
        axarr[row, 1].axis("off")
        axarr[row,1].imshow(img)
        if(name==True):
          title = "Ground Truth Image "+str(row+1)
        else:
          title = "Predicted Mask "+str(row+1)
        title_color = 'black'
        title = axarr[row, 1].set_title(title)
        plt.setp(title, color=title_color)
      else:
        img = Pred[row]
        axarr[row, 2].axis("off")
        axarr[row,2].imshow(img)
        title = "Predicted Image "+str(row+1)
        title_color = 'black'
        title = axarr[row, 2].set_title(title)
        plt.setp(title, color=title_color)

In [ ]:
img = Y[0]
# axarr[0, 1].axis("off")
plt.imshow(img)
cv2.imwrite(os.path.join("img.jpg"),img)


In [ ]:
X=[]
Y=[]
draw_over=[]
gt=[]
for ind in range(4):
  i = random.randint(0, len(train_ids)-10)
  X.append(X_train[i])
  gt.append(Y_train[i])
  Y.append(resize(np.squeeze(preds_train_t[i]), (1024, 1360), mode='constant', preserve_range=True))
  image = np.squeeze(preds_train_t[i])
  img_1=X_train[i].copy()
  lbl_0 = label(image) 
  props = regionprops(lbl_0)
  for prop in props:
    cv2.rectangle(img_1, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (255, 0, 0), 1)
  draw_over.append(img_1)

In [ ]:
X=[]
Y=[]
draw_over=[]
gt=[]
for ind in range(4):
  i = random.randint(0, len(train_ids)-10)
  X.append(X_train[i])
  gt.append(Y_train[i])
  Y.append(resize(np.squeeze(preds_train_t[i]), (1024,1360), mode='constant', preserve_range=True))
  image = np.squeeze(preds_train_t[i])
  img_1=X_train[i].copy()
  lbl_0 = label(image) 
  props = regionprops(lbl_0)
  for prop in props:
    cv2.rectangle(img_1, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (255, 0, 0), 1)
  draw_over.append(img_1)

In [ ]:
draw_sub_plots_3(4,3,X,Y,draw_over,True)

In [ ]:
X=[]
Y=[]
draw_over=[]
for ind in range():
  i = random.randint(0, len(test_ids))
  X.append(X_test[i])
  Y.append(resize(np.squeeze(preds_test_t[i]), (128, 128), mode='constant', preserve_range=True))
  image = np.squeeze(preds_test_t[i])
  img_1=X_test[i].copy()
  lbl_0 = label(image) 
  props = regionprops(lbl_0)
  for prop in props:
    cv2.rectangle(img_1, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (255, 0, 0), 1)
  draw_over.append(img_1)

In [ ]:
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
names=[]
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH 
    img = imread(path +  id_ )[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    names.append(id_)
    X_test[n] = img

In [ ]:
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
X=[]
Y=[]
draw_over=[]
for i in range(len(X_test)):
  X.append(X_test[i])
  Y.append(resize(np.squeeze(preds_test_t[i]), (1024, 1360), mode='constant', preserve_range=True))
  image = np.squeeze(preds_test_t[i])
  img_1=X_test[i].copy()
  lbl_0 = label(image) 
  props = regionprops(lbl_0)
  for prop in props:
    cv2.rectangle(img_1, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (0, 0, 255), 1)
  draw_over.append(img_1)

In [ ]:
for ind,img in enumerate(Y):
    print(os.path.join("Results/"+names[ind]+"_mask.jpg"))
    plt.imsave(os.path.join("Results/"+names[ind]+"_mask.jpg"),img)
    plt.imsave(os.path.join("Results/"+names[ind]+".jpg"),X[ind])
    plt.imsave(os.path.join("Results/"+names[ind]+"_bbox.jpg"),draw_over[ind])
    

In [ ]:
draw_sub_plots_3(4,3,X,Y,draw_over,False)